In [38]:
df_get = pd.read_csv('./toh_get.csv');
# print(df_get.describe)
set_get = set(df_get['source'])
# print(len(set_get));

df_use = pd.read_csv('./toh_use.csv');
# print(df_use.describe)
set_use = set(df_use['source'])
# print(len(set_use));

ss_all = set(list(set_get) + list(set_use));
print(ss_all)
print(len(ss_all))

{'顺序兑换活动活动赠', '竞技场结束奖励', '公会boss击杀奖励', '等级基金的邮件', '天马探索', '解锁升级公会科技', '升级公会订单', '酒馆活动活动赠', '等级限时礼包', '神器升级', '卡牌包扩容', '卡牌碎片合成', '地牢关卡掉落', '买竞技场门票', '工会商店刷新', '转换水晶', '新的黑市商店购买', '指定英雄升星活动赠', '巨龙之岛金苹果初始化', '跨服pvp排行榜奖励', 'battle pass买积分', '地牢买药水', '购买封印战斗次数', '竞技场获胜活动赠', '3v3竞技场的3选1', '先知置换-人类', '符文升级', '买转盘硬币', '爱心抽卡', '先知召唤-恶魔', '公会捐献', '英雄分解商店购买', 'xx商店购买', '每日任务领奖', '好友送心', '3选1远征', '先知召唤-天灾', '捐出卡牌碎片获得奖励', '低级抽卡', '高星置换-7星', '公会boss排名奖励', '瓦尔哈拉之邀活动赠', '巨龙之岛建筑产出', '钻石礼包', '扫荡', '点金手', '能量抽卡', '封印扫荡', '获得捐出的卡牌碎片', '装备附魔', '虚空普通战斗奖励', '巨龙之岛刷新boss', '先知召唤-人类', '地牢买商人', '节日礼包', '巨龙之岛重置建筑', '远征复活', '买爬塔体力', 'FRIEND_MAZE_BOX', '巨龙之岛买苹果', '藏宝图2活动赠', '重置公会科技', '地牢刷新职业', '公会签到', '封印战斗', '训练营领奖', '回收旅行骰子', '天马买耐久', '神圣商店购买', '卡牌分解', '购买封印扫荡次数', '先知置换-天灾', '符文分解', '符文解锁', '天马升级任务', '退款后扣除', '转盘次数活动赠', '封印之地商店购买', '装备合成', '解锁水晶', '高级抽卡', '挑战赛积分赛', '公会挖矿', '公会礼包打开', '卡牌赋能', '酒馆刷新', '转盘商店购买', '虚空击杀奖励', '高星置换-8星', '虚空排行奖励', 'REDEMPTION_CODE', 'FRIEND_MAZE_FIGHT', '英雄委托活动赠', 'vip礼包', '签到宝箱', '神器合成',

In [43]:
func_ = pd.read_csv('./func_system.csv');
sys = func_["system"];
ss_sys = set(sys);
print(len(ss_sys));
list(ss_sys)

33


['征战大厅',
 '成就系统',
 '英雄低到高合成',
 '副本系统',
 '酒馆',
 '爬塔',
 '碎片合成',
 '转盘',
 '新手养成',
 '工会',
 '月礼包',
 '远征',
 '运营处理',
 '竞技场',
 '成就',
 '龙岛',
 '月活动',
 '礼包',
 '周活动',
 '其他养成',
 '先知',
 '活跃活动',
 '抽卡',
 '好友',
 '分解功能',
 '装备',
 '卡牌',
 '封印之地',
 '个人信息',
 '符文书',
 '英雄',
 '商店',
 '地牢']

In [1]:
#从数据库获取数据 更新toh_get_ toh_use相关原始数据
import sys
import os
import codecs
from sys import argv
import pymysql
import pandas as pd

config = dict(host='125.124.0.156', user='chencheng', password='Cc$toh$db1', port= 3306,database = "toh", cursorclass=pymysql.cursors.DictCursor, charset='UTF8')

conn = pymysql.Connect(**config)
conn.autocommit(1)
cursor = conn.cursor()

dic = {};

def run_out():
    sql_ = '''select `desc`,system,channel from
(select channel,`desc` from
	(select * from (select distinct other, type from `toh_use_316`) result
		left join toh_via 
		on result.other = toh_via.id_) result2
left join toh_res on result2.type = toh_res.id) result3
left join toh_via_func on result3.channel = toh_via_func.`function`
    '''
    cursor.execute(sql_)
    all_ret = cursor.fetchall()
    out_data = pd.DataFrame(all_ret);
    print(out_data)
    out_data.to_csv("toh_use.csv", index=False);
    del out_data['channel'];
    new = out_data.drop_duplicates(subset=['desc','system'])
    new.to_csv("toh_use2.csv", index=False);


def run_get():
    sql_ = '''select `desc`,system,channel from
(select channel,`desc` from
	(select * from (select distinct other, type from `toh_get_316_copy`) result
		left join toh_via 
		on result.other = toh_via.id_) result2
left join toh_res on result2.type = toh_res.id) result3
left join toh_via_func on result3.channel = toh_via_func.`function`
    '''
    cursor.execute(sql_)
    all_ret = cursor.fetchall()
    out_data = pd.DataFrame(all_ret);
    print(out_data)
    out_data.to_csv("toh_get.csv", index=False);
    del out_data['channel'];
    new = out_data.drop_duplicates(subset=['desc','system'])
    new.to_csv("toh_get2.csv", index=False);

if __name__ == '__main__':
    '''
    main
    '''
    run_get();
    run_out();

          desc system             channel
0           金币    周活动           顺序兑换活动活动赠
1         时光之沙    周活动           顺序兑换活动活动赠
2           卡牌    周活动           顺序兑换活动活动赠
3     固定英雄卡牌碎片    周活动           顺序兑换活动活动赠
4           装备    周活动           顺序兑换活动活动赠
..         ...    ...                 ...
535    骰子 dice    周活动  登陆有奖(活动期间登录拿东西)活动赠
536        虚空石    周活动  登陆有奖(活动期间登录拿东西)活动赠
537  lemon_tea    周活动  登陆有奖(活动期间登录拿东西)活动赠
538  stopwatch    周活动  登陆有奖(活动期间登录拿东西)活动赠
539   egg_tart    周活动  登陆有奖(活动期间登录拿东西)活动赠

[540 rows x 3 columns]
                      desc system      channel
0                       钻石    周活动    顺序兑换活动活动赠
1                      灵魂石    周活动    顺序兑换活动活动赠
2                lemon_tea    周活动    顺序兑换活动活动赠
3                stopwatch    周活动    顺序兑换活动活动赠
4    horse searach stamina     龙岛         天马探索
..                     ...    ...          ...
142                    许愿币     转盘         使用转盘
143                     钻石     工会         创建公会
144                 进背包的宝箱    周活动          开宝箱
14

In [2]:
#从toh_get_ toh_use相关原始数据 整理出作图需要的out.csv数据
import networkx as nx
import pandas as pd

l_source = set({});
l_funct = set({})

df = pd.read_csv('./toh_use2.csv')
df.rename(columns={'desc': 'source', 'system' : 'target'}, inplace=True);
Graphtype = nx.DiGraph()
G = nx.from_pandas_edgelist(df, create_using=Graphtype)
list(G)

list(G.edges())
for node in list(G):
    nbrs = nx.neighbors(G, node)
#     print(node, list(set(nbrs)));
    if(len(set(nbrs)) == 0):
        l_funct.add(node);
    else:
       #是资源类型
        l_source.add(node);

df2 = pd.read_csv('./toh_get2.csv')
df2.rename(columns={'desc': 'target', 'system' : 'source'}, inplace=True);
Graphtype = nx.DiGraph()
G2 = nx.from_pandas_edgelist(df2, create_using=Graphtype)
list(G2.edges())
for node in list(G2):
    nbrs = nx.neighbors(G2, node)
#     print(node, list(set(nbrs)));
    if(len(set(nbrs)) == 0):
       #是资源类型
       l_source.add(node);
    else:
        l_funct.add(node);
       #是功能类型

# 只取某系统/某资源对应系统
U_all=nx.DiGraph()
U_all.add_edges_from(G.edges())
U_all.add_edges_from(G2.edges());
U_all.add_nodes_from(G.nodes())
U_all.add_nodes_from(G2.nodes())
U_all.remove_node('周活动');
U_all.remove_node('月活动');
U_all.remove_node('月礼包');
U_all.remove_node('礼包');

def out_csv(_U_all, out_path = "to_io_draw.csv"):
    df_out = pd.DataFrame([], columns=['id','name','shape','fill','refs']);
    i = 0;
    for node in list(_U_all):
        nbrs = nx.neighbors(_U_all, node)
        shape_ = "ellipse" if(node in l_source) else "rectangle"
        fill_ = "#b85450" if(node in l_source) else "#82b366"
        row = [node, node, shape_, fill_ ,','.join(list(set(nbrs)))]
        df_out.loc[i]=row
        i+=1;
    df_out.to_csv(out_path, index=False);

for s in l_source:
    subnodes = list(nx.all_neighbors(U_all, s)) + [s];
    print(subnodes)
    sub = U_all.subgraph(subnodes)
    print("edgess " + s, list(sub.nodes), list(sub.edges));    
    out_csv(sub, "./draw_io_out/source/to_io_draw_" + s + ".csv");

for s in l_funct:
    subnodes = list(nx.all_neighbors(U_all, s)) + [s];
    print(subnodes)
    sub = U_all.subgraph(subnodes)
    print("edgess " + s, list(sub.nodes), list(sub.edges));    
    out_csv(sub, "./draw_io_out/system/to_io_draw_sys_" + s + ".csv");
#     print(row)
#     new_row = pd.DataFrame(, columns=['id','name','refs']);
#     print(new_row)
#     df_out.append(new_row, ignore_index=True)



['征战大厅', 'egg_tart']
edgess egg_tart ['egg_tart', '征战大厅'] [('征战大厅', 'egg_tart')]
['竞技场', '地牢', '基础商店', '活跃活动', '酒馆', '竞技场', '竞技场门票']
edgess 竞技场门票 ['活跃活动', '竞技场门票', '地牢', '基础商店', '酒馆', '竞技场'] [('活跃活动', '竞技场门票'), ('竞技场门票', '竞技场'), ('地牢', '竞技场门票'), ('基础商店', '竞技场门票'), ('酒馆', '竞技场门票'), ('竞技场', '竞技场门票')]
['装备系统', '装备附魔书']
edgess 装备附魔书 ['装备系统', '装备附魔书'] [('装备附魔书', '装备系统')]
['封印之地', '封印之地', '封印之地扫荡次数']
edgess 封印之地扫荡次数 ['封印之地', '封印之地扫荡次数'] [('封印之地', '封印之地扫荡次数'), ('封印之地扫荡次数', '封印之地')]
['龙岛', '分解功能', '背包系统', '随机皮肤碎片']
edgess 随机皮肤碎片 ['背包系统', '分解功能', '随机皮肤碎片', '龙岛'] [('分解功能', '随机皮肤碎片'), ('随机皮肤碎片', '背包系统'), ('龙岛', '随机皮肤碎片')]
['milk']
edgess milk ['milk'] []
['工会', '新手养成', '基础商店', '先知之树', '转盘', '分解功能', '活跃活动', '远征', '爬塔', '神圣商店', '成就系统', '征战大厅', '背包系统', '运营处理', '工会', '固定英雄卡牌碎片']
edgess 固定英雄卡牌碎片 ['神圣商店', '工会', '活跃活动', '远征', '新手养成', '征战大厅', '基础商店', '运营处理', '固定英雄卡牌碎片', '分解功能', '背包系统', '成就系统', '先知之树', '爬塔', '转盘'] [('神圣商店', '固定英雄卡牌碎片'), ('工会', '固定英雄卡牌碎片'), ('活跃活动', '固定英雄卡牌碎片'), ('远征', '固定英雄卡牌碎片'), ('新手养成'

['地牢', '龙岛', '符文书', '活跃活动', '远征', '征战大厅', '符文书', '混沌石， 符文商店用']
edgess 混沌石， 符文商店用 ['活跃活动', '地牢', '远征', '混沌石， 符文商店用', '征战大厅', '符文书', '龙岛'] [('活跃活动', '混沌石， 符文商店用'), ('地牢', '混沌石， 符文商店用'), ('远征', '混沌石， 符文商店用'), ('混沌石， 符文商店用', '符文书'), ('征战大厅', '混沌石， 符文商店用'), ('符文书', '混沌石， 符文商店用'), ('龙岛', '混沌石， 符文商店用')]
['好友', '好友', '好友boss体力，新的3']
edgess 好友boss体力，新的3 ['好友boss体力，新的3', '好友'] [('好友boss体力，新的3', '好友'), ('好友', '好友boss体力，新的3')]
['竞技场', '新手养成', '基础商店', '先知之树', '成就系统', '酒馆', '先知之树', '先知祝福，英雄置换']
edgess 先知祝福，英雄置换 ['先知祝福，英雄置换', '新手养成', '基础商店', '酒馆', '成就系统', '先知之树', '竞技场'] [('先知祝福，英雄置换', '先知之树'), ('新手养成', '先知祝福，英雄置换'), ('基础商店', '先知祝福，英雄置换'), ('酒馆', '先知祝福，英雄置换'), ('成就系统', '先知祝福，英雄置换'), ('先知之树', '先知祝福，英雄置换'), ('竞技场', '先知祝福，英雄置换')]
['封印之地', '封印之地', '封印之地四色结晶']
edgess 封印之地四色结晶 ['封印之地', '封印之地四色结晶'] [('封印之地', '封印之地四色结晶'), ('封印之地四色结晶', '封印之地')]
['地牢', 'LOSTCITY_RECOVERY']
edgess LOSTCITY_RECOVERY ['地牢', 'LOSTCITY_RECOVERY'] [('地牢', 'LOSTCITY_RECOVERY')]
['工会', '征战大厅', 'stopwatch']
edgess stopwatch ['stopwatch'

NetworkXError: The node 月礼包 is not in the digraph.

In [106]:
test_dup = pd.read_csv("toh_get2.csv");
print(test_dup);
g = test_dup.groupby(['desc','system']).groups
print(len(g))
# test_dup.drop_duplicates();
# print(test_dup)

          desc system
0           金币    周活动
1         时光之沙    周活动
2           卡牌    周活动
3     固定英雄卡牌碎片    周活动
4           装备    周活动
..         ...    ...
535    骰子 dice    周活动
536        虚空石    周活动
537  lemon_tea    周活动
538  stopwatch    周活动
539   egg_tart    周活动

[540 rows x 2 columns]
283
